In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import sys
sys.path.insert(1, '../Data_exploration_notebooks')

import WranglerFunctions as wf

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
earnings = pd.read_csv('../Datasets/Financial_dataset/F3METL_Comp.csv', index_col=False, encoding = 'unicode_escape')
earnings = wf.F3Metl_Comp_Wrangler(earnings)
earnings

In [ ]:
RIO_earnings = earnings['RIO LN Equity']['EBITDA'].to_frame()
RIO_earnings

In [ ]:
mean = RIO_earnings['EBITDA'].rolling('90D', min_periods=1).mean()
std = RIO_earnings['EBITDA'].rolling('90D', min_periods=1).std()

RIO_earnings['EBITDA'].plot()
plt.show()
mean.plot()
plt.show()
std.plot()
plt.show()

In [ ]:
earnings['RIO LN Equity'].loc[earnings['RIO LN Equity']['EBITDA'].drop_duplicates().index].corr()

# LSTM

Code from website below:

https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

When rerunning code you have to restart the kernel and run from the start.

In [ ]:
inflation_original = pd.read_csv('../Datasets/Economic_dataset/Inflation.csv', index_col = False)
clean_inflation = wf.Inflation_Unemployment_Wrangler(inflation_original)
unemployment_original = pd.read_csv('../Datasets/Economic_dataset/Unemployment.csv', index_col = False)
clean_unemployment, _ = wf.Inflation_Unemployment_Wrangler(unemployment_original)

def getEBITDAUnique(df, companyName):
    earnings_unique = df[companyName]['EBITDA'].drop_duplicates()
    earnings_unique = pd.DataFrame(earnings_unique)
    earnings_unique = earnings_unique.dropna()
    earnings_unique = earnings_unique.reset_index()
    return earnings_unique

def closest_date(dateList, date, start, end):
    if start > end:
        return (start + end) // 2

    mid = (start + end) // 2
    if date == dateList[mid]:
        return mid

    if date < dateList[mid]:
        return closest_date(dateList, date, start, mid-1)
    else:
        return closest_date(dateList, date, mid+1, end)

def add_inflation(df, inflation):
    df['Inflation Rate'] = np.nan
    i = 0
    for i in range(len(df)):
        date = df['Dates'][i]
        closestInflationDate = closest_date(inflation['Date'], date, 0, len(inflation['Date']))
        df.loc[i, 'Inflation Rate'] = inflation['Inflation Rate'][closestInflationDate]
    return df

def add_Unemployment(df, unemployment):
    df['Unemployment'] = np.nan
    i = 0
    for i in range(len(df)):
        date = df['Dates'][i]
        closestDate = closest_date(unemployment['Date'], date, 0, len(unemployment['Date']))
        df.loc[i, 'Unemployment'] = unemployment['Unemployment Rate'][closestDate]
    return df

In [ ]:
RIO_unique = getEBITDAUnique(earnings, "RIO LN Equity")
add_Unemployment(RIO_unique, clean_unemployment)
add_inflation(RIO_unique, clean_inflation[0])
RIO_unique.index = RIO_unique["Dates"]
RIO_unique = RIO_unique.drop("Dates", axis = 1)
RIO_unique

In [ ]:
groups = [0, 1, 2]
i = 1
values = RIO_unique.values
# plot each column
plt.figure()
for group in groups:
    plt.subplot(len(groups), 1, i)
    plt.plot(values[:, group])
    plt.title(RIO_unique.columns[group], y=0.5, loc='right')
    i += 1
plt.show()

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
    
# Lag is number of datapoints it looks previously
lag = 1
reframed = series_to_supervised(scaled, lag, 1)
reframed.drop(reframed.columns[[9, 10, 11]], axis=1, inplace=True)
reframed

In [ ]:
# split into train and test sets
values = reframed.values
n_train_hours = 30
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
lstm = LSTM(50, input_shape = (train_X.shape[1], train_X.shape[2]))
model.add(lstm)
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

## Plotting forecast

In [ ]:
yhatTrain = model.predict(train_X)
plt.plot(range(len(yhatTrain)), yhatTrain, c = 'b', marker = 'x')
plt.plot(range(len(train_y)), train_y, c = 'r', marker = 'o')

plt.plot(range(len(yhatTrain), len(RIO_unique) - 1), yhat, c = 'b', marker = 'x')
plt.plot(range(len(train_y), len(RIO_unique) - 1), test_y, c = 'r', marker = 'o')
plt.axvline(len(train_y))
plt.show()